In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
dfb = pd.read_csv("benign_data.csv", index_col='Id')
dfm = pd.read_csv("malware_data.csv", index_col='Id')

all_perm_uni = list(dfb.columns)
apps_ben = list(dfb.index)
apps_mal = list(dfm.index)

In [3]:
sorted_inc = []
sorted_dec = []
for k in range(0, 11):
    rank = pd.Series()
    for j in all_perm_uni:
        sumB = 0
        sumM = 0
        for i in apps_ben:
            sumB += dfb.loc[i,j]
        for i in apps_mal:
            sumM += dfm.loc[i,j]
        
        # Use permisssions with support 0%, 5%, 10%, 15%, ..., 50%
        if sumB >= (k)*0.05*len(apps_ben) and sumM >= (k)*0.05*len(apps_mal):
            fB = sumB / len(apps_ben)
            fM = sumM / len(apps_mal)
            rank[j] = (fM - fB)
    
    inc_rank = rank.sort_values()
    dec_rank = rank.sort_values(ascending=False)
    sorted_inc.append(inc_rank)
    sorted_dec.append(dec_rank)

In [11]:
# print(sorted_dec[0][:20])

In [42]:
accuracy_svm = []
accuracy_nb = []
accuracy_rf = []
for k in range(0, 11):
    accuracy = []
    # index = list(sorted_inc[k].index)
    index = list(sorted_dec[k].index)
    x = 0.0
    i = 1
    while i < len(index)+1:
        ben = dfb[index[0:i]]
        mal = dfm[index[0:i]]
        yb = pd.Series(np.zeros(len(ben.index)))
        ym = pd.Series(np.ones(len(mal.index)))
        
        X = pd.concat([ben, mal])
        y = pd.concat([yb, ym])
        X.fillna(0)
        
        # Splitting the dataset into the Training set and Test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

        # Creating and training classifiers
        #classifier1 = RandomForestClassifier(criterion='entropy', random_state = 0)
        #classifier2 = GaussianNB()
        classifier3 = SVC(kernel='rbf', random_state=0)
        
        #classifier1.fit(X_train, y_train)
        #classifier2.fit(X_train, y_train)
        classifier3.fit(X_train, y_train)

        # Predicting the Test set results
        y_pred = classifier3.predict(X_test)
        a = accuracy_score(y_pred, y_test)
        if x < a:
            accuracy.append(a)
            x = a
            i = i+1
        else:
            index.pop(i-1)
        '''accuracy1.append(accuracy_score(y_pred, y_test))
        y_pred = classifier2.predict(X_test)
        accuracy2.append(accuracy_score(y_pred, y_test))
        y_pred = classifier3.predict(X_test)
        accuracy3.append(accuracy_score(y_pred, y_test))'''
        
    #accuracy_rf.append(index)
    #accuracy_nb.append(index)
    accuracy_svm.append(index)
    
    #accuracy_rf.append(accuracy)
    #accuracy_nb.append(accuracy)
    accuracy_svm.append(accuracy)

In [43]:
#df_rf = pd.DataFrame(accuracy_rf).transpose();
#df_nb = pd.DataFrame(accuracy_nb).transpose();
df_svm = pd.DataFrame(accuracy_svm).transpose();

#df_rf.to_csv('accuracy_rf_3d.csv');
#df_nb.to_csv('accuracy_nb_3d.csv');
df_svm.to_csv('accuracy_svm_3d.csv');

In [25]:
# df_rf.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,WAKE_LOCK,0.65014,WAKE_LOCK,0.65014,WAKE_LOCK,0.65014,WAKE_LOCK,0.65014,WAKE_LOCK,0.65014,...,WAKE_LOCK,0.65014,WAKE_LOCK,0.65014,ACCESS_NETWORK_STATE,0.583799,ACCESS_NETWORK_STATE,0.583799,ACCESS_NETWORK_STATE,0.583799
1,USE_CREDENTIALS,0.659916,ACCESS_NETWORK_STATE,0.660615,ACCESS_NETWORK_STATE,0.660615,ACCESS_NETWORK_STATE,0.660615,ACCESS_NETWORK_STATE,0.660615,...,ACCESS_NETWORK_STATE,0.660615,ACCESS_NETWORK_STATE,0.660615,ACCESS_WIFI_STATE,0.613827,ACCESS_WIFI_STATE,0.613827,None,None
2,ACCESS_NETWORK_STATE,0.670391,CAMERA,0.665503,WRITE_EXTERNAL_STORAGE,0.683659,WRITE_EXTERNAL_STORAGE,0.683659,WRITE_EXTERNAL_STORAGE,0.683659,...,WRITE_EXTERNAL_STORAGE,0.683659,WRITE_EXTERNAL_STORAGE,0.683659,None,None,None,None,None,None
3,CAMERA,0.674581,BLUETOOTH,0.666201,ACCESS_COARSE_LOCATION,0.688547,ACCESS_COARSE_LOCATION,0.688547,ACCESS_COARSE_LOCATION,0.688547,...,ACCESS_COARSE_LOCATION,0.688547,ACCESS_WIFI_STATE,0.685754,None,None,None,None,None,None
4,MANAGE_ACCOUNTS,0.680168,WRITE_EXTERNAL_STORAGE,0.687849,SYSTEM_ALERT_WINDOW,0.715084,READ_CONTACTS,0.731145,RECEIVE_BOOT_COMPLETED,0.715782,...,RECEIVE_BOOT_COMPLETED,0.715782,READ_PHONE_STATE,0.75838,None,None,None,None,None,None


In [ ]:
9